# Stock --> Positition --> Portfolio

In [ ]:
import requests as r
from bs4 import BeautifulSoup
from dataclasses import dataclass
from tabulate import tabulate

In [ ]:
def get_fx_to_usd(currency, price):
  url = f"https://www.google.com/finance/quote/{currency}-USD"
  resp = r.get(url)
  soup = BeautifulSoup(resp.content, "html.parser")
  fx_div = soup.find("div", attrs={"data-last-price": True})
  fx = float(fx_div["data-last-price"])
  price_usd = round(fx*price, 2)
  return price_usd

In [ ]:
def get_price_information(ticker, exchange):
  url = f"https://www.google.com/finance/quote/{ticker}:{exchange}"
  params = {"hl": "en"}
  resp = r.get(url, params=params)
  soup = BeautifulSoup(resp.content, "html.parser")
  price_div = soup.find("div", attrs={"data-last-price": True})
  price = float(price_div["data-last-price"])
  currency = price_div["data-currency-code"]
  usd_price = price
  if currency!="USD":
    usd_price = get_fx_to_usd(currency, price)

  return {
      "ticker": ticker,
      "exchange": exchange,
      "currency": currency,
      "price": price,
      "usd_price": usd_price
  }



In [ ]:
def get_fx_to_usd(currency, price):
  url = f"https://www.google.com/finance/quote/{currency}-USD"
  resp = r.get(url)
  soup = BeautifulSoup(resp.content, "html.parser")
  fx_div = soup.find("div", attrs={"data-last-price": True})
  fx = float(fx_div["data-last-price"])
  price_usd = round(fx*price, 2)
  return price_usd

In [ ]:
@dataclass
class Stock:
  ticker: str
  exchange: str
  price: float = 0
  currency: str = "USD"
  usd_price: float = 0

  def __post_init__(self):
    price_info = get_price_information(self.ticker, self.exchange)

    if price_info["ticker"] == self.ticker:
      self.price = price_info["price"]
      self.currency = price_info["currency"]
      self.usd_price = price_info["usd_price"]


In [ ]:
@dataclass
class Position:
  stock: Stock
  quantity: int = 0


In [ ]:
@dataclass
class Portfolio:
  positions: list[Position]

  def get_total_value(self):
    total_value = 0

    for position in self.positions:
      total_value += position.quantity * position.stock.usd_price

    return total_value



In [ ]:
def display_portfolio_summary(portfolio):
  if not isinstance(portfolio, Portfolio):
    raise TypeError("Please provide an instance of the portfolio type")
  portfolio_value = portfolio.get_total_value()

  position_data = []

  for position in sorted(portfolio.positions,
                         key=lambda x:x.quantity * x.stock.usd_price,
                         reverse=True):
    position_data.append([
        position.stock.ticker,
        position.stock.exchange,
        position.quantity,
        # position.stock.currency,
        position.stock.usd_price,
        position.quantity * position.stock.usd_price,
        position.quantity * position.stock.usd_price / portfolio_value * 100
        ])

  print(tabulate(position_data,
                 headers=["Ticker", "Exchange", "Quantity", "Price", "Market Value", "% Allocation"],
                 tablefmt="psql",
                 floatfmt=".2f"))
  print(f"Total portfolio value: ${portfolio_value:,.2f}")



In [ ]:
shop = Stock("SHOP", "TSE")
msft = Stock("MSFT", "NASDAQ")
googl = Stock("GOOGL", "NASDAQ")
portfolio = Portfolio([
    Position(shop, 10),
    Position(msft, 2),
    Position(googl, 30)])
print(portfolio.get_total_value())
display_portfolio_summary(portfolio)

5440.2300000000005
+----------+------------+------------+---------+----------------+----------------+
| Ticker   | Exchange   |   Quantity |   Price |   Market Value |   % Allocation |
|----------+------------+------------+---------+----------------+----------------|
| GOOGL    | NASDAQ     |         30 |  134.19 |        4025.55 |          74.00 |
| MSFT     | NASDAQ     |          2 |  369.24 |         738.48 |          13.57 |
| SHOP     | TSE        |         10 |   67.62 |         676.20 |          12.43 |
+----------+------------+------------+---------+----------------+----------------+
Total portfolio value: $5,440.23
